# What to expect

In this notebook, we will run some of the initial quality filtering and mapping with downsampled data from the *Schistosoma* dataset. If you need to revisit the presentation where we introduced RNAseq and the example dataset, you can find it on Learn, in the "Workshop 1" folder. You will also find the original paper where this dataset was published, and some review articles about RNA-Seq analysis methods. These are not compulsory reading, but may be of interest and are worth a look.

In places we will provide the code to run an analysis step first, and then describe what it is doing. **Some of these steps may take a few minutes, and this time can be spent answering the questions, reading ahead or understanding the process as it runs**.

## The command line

Whilst the first and second year courses focused on teaching coding in python, another key skill in biology is to run specialized existing software. Some of these can be installed as python modules, but many real-world tools are run "on the command line". This means that they run like an application or program, but the user types commands in a "shell" or "terminal" instead of clicking/swiping in an interactive window. Within the notebook environment, these commands can be run in 3 ways:

* adding `%%bash` to the top of a cell
* adding a `!` to the start of the command in a cell
* (sometimes) if the command can only be interpreted in bash, jupyter sometimes doesn't need to be told. There is a really excellent (short) primer to the command line which can be found [here](https://cyverse-leptin-rna-seq-lesson-dev.readthedocs-hosted.com/en/latest/section-3.html) and gives a description of the most common commands.

During this (and subsequent) workshops, we will combine code that is written on the command line (for which the full command will be provided) and questions which will require you to use the python coding you have already learnt.

In [ ]:
# Let's start by installing dependancies we will use (takes a while, read ahead while running)
! pip install --quiet biopython
! mamba install --yes --quiet bioconda::fastqc
! mamba install --yes --quiet bioconda::trim-galore
! mamba install --yes --quiet bioconda::star

<div class="alert alert-block alert-info">

In the code above:

`mamba` - Package manager. We use it to install software. `conda` can be used instead of `mamba`.

`--yes` - confirm that we want to install all the dependencies

`--quiet` - do not show extra output

# The raw data

The data is stored in `data/Schistosoma_mansoni`. Here we find four elements:

1. `README` file - contains basic information about the data in this folder.
2. `list_ids` file - contains the ids for the reads in this dataset. Each id corresponds to one sample (eg Schistosomula 3h post infection, replicate 1)
3. `reference` folder - contains the reference genome and transcriptome
4. `subsampled` folder - contains the raw data file for the subset of sequences we have taken for this workshop

Let's see what we have in this examples dataset. 

In [ ]:
# Print the list of ids for this example dataset
! cat data/Schistosoma_mansoni/list_ids.txt

<div class="alert alert-block alert-info">
In the code above:

`cat` - display the contents of a file


# FASTQ structure

<div class="alert alert-block alert-warning">

Questions:
1. The raw data files look like this `<accession>_<1|2>.fastq.gz`. What does the ".gz" in the file name mean?
2. Pick one of the files and open it. What does it look like? Define the [FASTQ format](https://en.wikipedia.org/wiki/FASTQ_format)?

<details>
<summary><i>Hint</i></summary>

1. ".gz" is what is known as a file extension.
2. To view the file you first need to uncompress it. In Data Exploration (week 1, class 2) you uncompressed ".gz" files using `gunzip`. This time you want to save (`keep`) the uncompressed version of the file too. Once you have created the uncompressed version, you can open by double clicking in the file browser on the left hand side, or can download it and open it with a text editor.

</details>

<div class="alert alert-block alert-success">
Answers:



For standard file formats for biological data (like FASTQ), there are often tools or librarys which make it easier to interract with them. These usually check that the file is formatted as expected. They also make it much easier to find information in the file without having to look up the exact structure of the file format yourself. A commonly used python module is [SeqIO](https://biopython.org/wiki/SeqIO) from the python library [biopython](https://biopython.org/).

In your investigations on the FASTQ format you will have seen that scores are used to indicate how likely it is that a base reported in a sequencing read is in error. This is the [Phred score](https://learn.gencore.bio.nyu.edu/ngs-file-formats/quality-scores/). Now let's use the biopython library to investigate the sequence qualities in this file. 

For each read in a fastq file, SeqIO creates a [record](https://biopython.org/wiki/SeqRecord) object, with information about the read. The following code prints the record and Phred score for the first sequence in our FASTQ file.

In [ ]:
from Bio import SeqIO

for record in SeqIO.parse("data/Schistosoma_mansoni/subsampled/ERR022872_1.fastq", "fastq"):
    print(record)
    print(record.letter_annotations["phred_quality"])
    break

<div class="alert alert-block alert-info">

In the code above:

`SeqIO.parse("file name","file format")` - is a function that reads the file and gives the desired output

`break` - is needed to stop the loop, otherwise it would do it for every single read in the file, which is not what we want just now


<div class="alert alert-block alert-warning">

3. Choose one of the FASTQ files and use a for loop to find out what the highest and lowest Phred scores are in your chosen file.

<details>
<summary><i>Hint</i></summary>

For each record you need to:
 - find out the maximum and minimum Phred score in the record
 - update the variables

</details>

In [ ]:
#First define variables. All Phred scores will be bigger than 0 and less than 1000


#Write the loop

    
#Print the output



# Quality Control

Hopefully in the previous exercises it has become clear that sequencing DNA is not error-free - each base of the sequence is read with a degree of uncertainty and error - and that our data definitely has some low quality reads (and parts of reads). Each sequencing machine/method results in different error profiles. To improve our analysis we first want to filter the lowest quality reads. One tool commonly used to profile the amounts of error is [FastQC](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/). 

In [ ]:
# Create the output directory
! mkdir -p analysis/Schistosoma_mansoni/qc/

<div class="alert alert-block alert-info">

In the code above:

`mkdir` - command to create a new folder

`-p` - flag to create nested folders

Now let's do the FASTQC for each file (once the first one is complete, you can answer the questions while the others are running).

In [ ]:
%%bash
for accession in $(cat data/Schistosoma_mansoni/list_ids.txt)
do 
    fastqc data/Schistosoma_mansoni/subsampled/$accession*.fastq.gz --noextract -o analysis/Schistosoma_mansoni/qc
done


<div class="alert alert-block alert-info">

In the code above:

`for X in Y; do ... done` - this the structure for a loop in bash; we are asking that for each X (in this case accession) in Y (in this case the list of ids), the program does something (in this case fastqc)

`--noextract` - Do not uncompress the output file after creating it

`-o` - create all output files in the output directory specified next

<div class="alert alert-block alert-warning">

Open the output directory. You will see that two new files have been created for each read. Open one of the html files and have a look. [Here](https://hbctraining.github.io/Training-modules/planning_successful_rnaseq/lessons/QC_raw_data.html) you can find guidance on what each graph means. 

Questions:

4. Is there a pattern to where the errors occur in these reads?
5. Are there any overrepresented sequences? What are they from?

<div class="alert alert-block alert-success">
    
Answers:



# Trimming

When we generate millions of reads in a sequencing experiment, we are able to average multiple observations at the same location. However if there are portions of the reads which are low quality these may still affect the average and so we want to remove these regions. As you will have seen in the previous section, the adaptor sequences used to generate the DNA library may also be over-represented in the reads and could cause contamination.

The next step in an RNA-Seq analysis is therefore to trim poor quality regions and adaptor sequences. The vast majority of adapter sequences have already been removed but sometimes partial adapter sequences can remain in a small percentage of reads as a consequence of imperfect trimming (e.g. because of sequencing errors).  [TrimGalore](https://github.com/FelixKrueger/TrimGalore/blob/master/Docs/Trim_Galore_User_Guide.md) is a popular software for trimming sequencing reads. Again this is a slow step, so **take a look at the questions while it is running**.

In [ ]:
%%bash
for accession in $(cat data/Schistosoma_mansoni/list_ids.txt)
do
    trim_galore \
      data/Schistosoma_mansoni/subsampled/$accession*.fastq.gz \
      --paired \
      --output_dir analysis/Schistosoma_mansoni/qc/ \
      --basename $accession \
      --no_report_file \
      --fastqc
done

<div class="alert alert-block alert-warning">

Question:

6. What do the following elements of the code above mean (Use the TrimGalore documentation to find the information)?

`--paired`

`--fastqc`

`--basename`

7. Chose a read file and compare the FastQC report before trimming to the FastQC report after adaptor trimming. What are the improvements to the data quality after trimming? Are there any remaining warnings?

<div class="alert alert-block alert-success">
    
Answers:



# Mapping to the reference

Now that we are happy with the quality of the reads and have removed adapters, we can map our sequences to the genome. This allows us to identify which genes each read came from, and which genes were <i>expressed</i> in our samples in the form of transcripts. 

<figure>
    <img src="https://www.annualreviews.org/docserver/ahah/fulltext/biodatasci/2/1/bd020139.f4_thmb.gif">
</figure>

In some organisms including *Plasmodium*, expressed transcripts may be generated by splicing together non-contiguous exons from the genome (others such as *Trypanosoma* do not). To handle this, we can either use a splice-aware mapper to align reads across splice junctions, or we can map directly against panels of known transcripts. In this example we are going to use [STAR](https://academic.oup.com/bioinformatics/article/29/1/15/272537) to perform splice-aware alignment to the reference genome fasta. The manual can be found [here](https://github.com/alexdobin/STAR/blob/master/doc/STARmanual.pdf). 


<div class="alert alert-block alert-danger">

This code will be run during the class, but uses more disk space than is available in Noteable. If you want to try it yourself, it will probably work on your personal computer. 

In [ ]:
%%bash
mkdir -p analysis/Schistosoma_mansoni/star/ref

# unzip the reference files
#gunzip data/Schistosoma_mansoni/reference/schistosoma_mansoni.PRJEA36577.WBPS19.annotations.gtf.gz 
#gunzip data/Schistosoma_mansoni/reference/schistosoma_mansoni.PRJEA36577.WBPS19.genomic.fa.gz

# first we need to index the reference
mkdir -p analysis/Schistosoma_mansoni/star/ref
  STAR --runThreadN 4 \
    --runMode genomeGenerate \
    --genomeDir analysis/Schistosoma_mansoni/star/ref \
    --genomeFastaFiles data/Schistosoma_mansoni/reference/*genomic.fa \
    --sjdbGTFfile data/Schistosoma_mansoni/reference/*.gtf \
    --sjdbOverhang 75 \
    --genomeSAindexNbases 11

<div class="alert alert-block alert-info">

In the code above:

`--runMode genomeGenerate` - directs STAR to run genome indexing

`--genomeDir /path/to/genomeDir` - specifies where to store the index

`--genomeFastaFiles /path/to/genome/fasta` - provides the reference genome

`--sjdbGTFfile /path/to/annotations.gtf` - provides the coordinates of splice junctions in the reference genome

`--sjdbOverhang ReadLength-1` - this specifies the length of sequences to include in the splice junctions database

We can now use this index to align each pair of readfiles against the reference.

In [ ]:
%%bash
for accession in $(cat data/Schistosoma_mansoni/list_ids.txt)
do
    mkdir -p analysis/Schistosoma_mansoni/star/$accession
    
    STAR \
      --genomeDir analysis/Schistosoma_mansoni/star/ref \
      --runThreadN 4 \
      --readFilesIn <(gunzip -c analysis/Schistosoma_mansoni/qc/$accession*.fq.gz) \
      --outFileNamePrefix analysis/Schistosoma_mansoni/star/$accession/$accession \
      --outSAMtype BAM SortedByCoordinate \
      --limitBAMsortRAM 4000000 \
      --outSAMattributes Standard \
      --quantMode TranscriptomeSAM GeneCounts
done

<div class="alert alert-block alert-info">

In the code above:

`<(gunzip -c reads.fq.gz)` - this uncompresses the read sequences to input to STAR which does not support compressed files

`--outSAMtype BAM SortedByCoordinate` - sort and compress the output

`--outSAMattributes Standard` - include some standard count information in the output file

<div class="alert alert-block alert-warning">

Question:

8. Using the STAR manual, what outputs are generated using the flags `--quantMode TranscriptomeSAM GeneCounts`?

9. Find a python library which can load a SAM file

<div class="alert alert-block alert-success">

Answers:


We will be running STAR on each of the full datasets and will make the mapped read files available for the next class.

# Extension

There also exist several methods for transcript abundance quantification using `pseudo-alignment`. These methods don't fully line up reads against the reference genome or transcript sequences, but instead count the occurance of substrings of these transcripts and use this to estimate transcript abundances. 

One example of this method is [Kallisto](https://www.nature.com/articles/nbt.3519)

In [ ]:
! mamba install --yes --quiet bioconda::kallisto=0.48

In [ ]:
%%bash

mkdir -p analysis/Schistosoma_mansoni/kallisto/

kallisto index --index=analysis/Schistosoma_mansoni/kallisto/smansoni data/Schistosoma_mansoni/reference/schistosoma_mansoni.PRJEA36577.WBPS19.mRNA_transcripts.fa.gz

for accession in $(cat data/Schistosoma_mansoni/subsampled/list_ids.txt)
do
    kallisto quant --threads=2 \
      --index=analysis/Schistosoma_mansoni/kallisto/smansoni \
      --output-dir=analysis/Schistosoma_mansoni/kallisto \
      --gtf=data/Schistosoma_mansoni/reference/schistosoma_mansoni.PRJEA36577.WBPS19.annotations.gtf.gz \
      analysis/Schistosoma_mansoni/qc/"$accession"_1.trimmed.fastq.gz analysis/Schistosoma_mansoni/qc/"$accession"_2.trimmed.fastq.gz
done

<div class="alert alert-block alert-warning">

Question:

10. Give 2 differences between the methods used above by STAR and Kallisto to quantify transcript abundances

<div class="alert alert-block alert-success">

Answers:

